1. ## 길이 슬라이딩 + overlap

	손실 없음, 안정적
	문장 중간 잘림
	논문/의학 증례

2. ##  문장 기반 Sentence-based Chunking  -> kss, spacy, NLTK 문장분리기 
(규칙 기반 + 문장 분리기)

    ✔ 언제 좋음
	•	자연어 설명이 많은 문서
	•	문장 중심 구조(블로그, 설명서, 리포트, 교과서 등)

✔ 단점
	•	문장 분리가 느릴 수 있음
	•	일부 어려운 문장 구조에서 분리가 어색할 수 있음
	
		##kss 내부 동작 구조(공식 문헌 기반)
			•	한국어 종결 어미 규칙
			•	마침표/따옴표/괄호 조합에 대한 룰
			•	숫자/약어/리스트 표기 예외 처리
			•	“다.”/“요.”/“했다.” 패턴 학습
			•	문장 경계 후보를 놓고 통계 모델이 확률 판단

		즉:

		kss의 핵심 목표 = “문장이 어디서 끝나는지” 예측

		그리고 이 과정에서 길이, 의미, 슬라이딩 전혀 안 씀.

3. 문단 기반
    ✔ 특징
	•	빈 줄(\n\n) 기준으로 문단을 쪼개기
	•	문단 자체가 의미 덩어리라 RAG 검색 품질 고정
	•	문서 구조가 깨끗한 경우엔 최선
	•	보통 PDF → 텍스트 변환형 문서에서 정답

✔ 언제 좋음
	•	교과서, 보고서, 매뉴얼, 논문 (문단 명확한 경우)

✔ 단점
	•	너의 데이터처럼 문단이 거의 없는 경우 → 1덩어리로 나옴
	•	1문단이 너무 길면 잘라야 함 → 손실 가능성


4. 제목/헤더 기반
    ✔ 특징
	•	“제목/소제목”을 감지하여 섹션 단위로 먼저 split
	•	예:
	•	### 1. Introduction
	•	### Case 2 – Diagnosis
	•	2) 검사항목
	•	- 증상 등
	•	각 섹션 안에서 다시 길이 기준 슬라이딩 적용
	•	구조 + 내용 둘 다 보존

✔ 언제 좋음
	•	규칙 있는 텍스트 (매뉴얼, 보고서, 연구 문서, 메뉴판 등)
	•	제목 패턴이 명확한 JSON 라벨 형식

✔ 단점
	•	제목 패턴을 맞추지 못하면 split 실패
	•	의학 증례처럼 “문단이 바로 본문으로 시작”하는 케이스는 구조가 덜함

    
## 5. 의미 기반(semantic)
 

 1,2 순으로 적절하다.
 → 의미가 급격하게 바뀌는 지점을 자동 감지해 split

✔ 특징
	•	문장을 임베딩해서
	•	이전 문장과 코사인 유사도 급하락 시 “새 청크 시작”
	•	“토픽 전환 포인트”를 자동으로 인식
	•	대규모 문서에서도 의미 일관성을 보장하는 최첨단 방식
	•	LLM과의 궁합 완벽함

✔ 언제 좋음
	•	학술자료 / 여러 증례가 연속된 문서 / 매뉴얼 챕터 등
	•	의미 덩어리를 스스로 이어야 하는 정교한 RAG 구축 시

✔ 단점
	•	가장 느림
	•	sentence transformer 필요
	•	구현 난이도 ↑

- 기본: 두 줄 이상 개행(빈 줄) 기준 분리 (r"\n{2,}")
#    - 너무 짧은/긴 조각 필터링 -> 실제 문서에서 손실률 너무큼


In [22]:
import os
from pathlib import Path

SAVE_DIR = Path("./chunk_preview_output")
SAVE_DIR.mkdir(exist_ok=True)

def save_text(filename: str, content: str):
    path = SAVE_DIR / filename
    with open(path, "w", encoding="utf-8") as f:
        f.write(content)
    return str(path)
def chunk_v1_paragraph(text, max_chunk_chars=1200, min_len=30):
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    raw_chunks = [c.strip() for c in re.split(r"\n{2,}", text)]
    clean = []
    for c in raw_chunks:
        if not c or len(c) < min_len:
            continue
        if len(c) > max_chunk_chars:
            c = c[:max_chunk_chars]
        clean.append(c)
    return clean
def debug_preview_and_save1(
    data_dir: Path = DATA_DIR,
    text_field: str = "disease",
    max_chunk_chars: int = 1200,
    min_len: int = 30,
    max_files: int = 3,
    max_chunks_per_file: int = 3,
):
    pattern = str(data_dir / "**" / "*.json")
    file_paths = glob.glob(pattern, recursive=True)

    print(f"[debug] JSON 파일 {len(file_paths)}개 중 {max_files}개 미리보기\n")

    for fp in file_paths[:max_files]:
        print("#" * 80)
        print(f"[FILE] {fp}")

        for obj in _iter_json(fp):
            text = str(obj.get(text_field, "")).strip()
            title = obj.get("title", "N/A")
            dept  = obj.get("department", "N/A")

            print(f"  - title      : {title}")
            print(f"  - department : {dept}")
            print(f"  - text len   : {len(text)} chars")

            if not text:
                print("  (⚠️ disease 비어 있음)\n")
                continue

            # 💾 원본 저장
            original_name = Path(fp).stem + "_original.txt"
            save_text(original_name, text)

            chunks = chunk_v1_paragraph(text, max_chunk_chars=max_chunk_chars, min_len=min_len)

            # 💾 각 청크 저장
            chunk_paths = []
            for i, ch in enumerate(chunks):
                chunk_name = f"{Path(fp).stem}_chunk_{i}.txt"
                p = save_text(chunk_name, ch)
                chunk_paths.append(p)

            print(f"  → 청크 {len(chunks)}개 생성")
            print(f"  → 원본 저장: {original_name}")
            print(f"  → 청크 저장: {len(chunk_paths)}개 (예: {chunk_paths[:1]})\n")

            # 🖥️ Notebook 전/후 비교 출력
            print("-" * 60)
            print("[원본 text] (앞 300자)")
            print(text[:300] + ("...\n" if len(text) > 300 else "\n"))

            print("[청크 0] (앞 300자)")
            if chunks:
                print(chunks[0][:300] + ("...\n" if len(chunks[0]) > 300 else "\n"))

            for i, ch in enumerate(chunks[:max_chunks_per_file]):
                print("-" * 60)
                print(f"[CHUNK {i}] len={len(ch)}")
                print(ch[:400] + ("\n...\n" if len(ch) > 400 else "\n"))
                
debug_preview_and_save1(
    data_dir=DATA_DIR,
    text_field="disease",
    max_files=2,
    max_chunks_per_file=3,
)

[debug] JSON 파일 212개 중 2개 미리보기

################################################################################
[FILE] /Users/test/응 과제해/자연어처리/rag/data/TS_말뭉치데이터_안과/0e8bca31-e151-11ef-4bb1-00155dced605.json
  - title      : 40 .진돗개에서의 고니오디스게네시스 관련 녹내장
  - department : 안과
  - text len   : 4526 chars
  → 청크 1개 생성
  → 원본 저장: 0e8bca31-e151-11ef-4bb1-00155dced605_original.txt
  → 청크 저장: 1개 (예: ['chunk_preview_output/0e8bca31-e151-11ef-4bb1-00155dced605_chunk_0.txt'])

------------------------------------------------------------
[원본 text] (앞 300자)
40 .진돗개에서의 고니오디스게네시스 관련 녹내장
10세 된 온전한 암컷 진돗개 한 마리가 1주일 동안 결막 충혈과 눈 분비물의 병력을 가지고 내원했습니다. 왼쪽 눈. 위협 반응, 눈부심 반사, 직접 동공 빛 반사가 없었하였다. 슬릿 램프 생체 현미경 각막 부종, 섬모 홍조, 수성 발적이 드러났하였다. 안구 내압은 68 mmHg였습니다. 이용 가능한 자료에 따르면, 녹내장과 포도막염 진단이 내려졌습니다. 안압 상승에 따른 시신경 손상. 사례 보고 미주리주 콜럼비아 65211, 미국 고니오디즈게네시스(goniodysgenesis), ...

[청크 0] (앞 300자)
40 .진돗개에서의 고니오디스게네시스 관련 녹내장
10세 된 온전한 암컷 진돗개 한 마리가 1주일 동안 결막 충혈과 눈 분비물의 병력을 가지고 내원했습니다. 왼쪽 눈. 위협 반응, 눈부심 반사, 직접 동공 빛 반사

<!-- 2.📐 알고리즘 구조
	1.	텍스트 개행 정규화
→ Windows/macOS 등 개행 차이를 제거하고 \n으로 통일
	2.	빈 줄(\n\n) 기준으로 문단을 1차 분리
→ 긴 문서라도 최소한의 문단 구조는 유지
	3.	각 문단에 대해:
	•	길이가 1200자 이하면 그대로 사용
	•	길이가 1200자 초과면
→ 여러 개의 1200자 청크로 잘라냄
→ 각 청크는 200자 정도 앞 청크의 끝부분을 겹쳐서(overlap) 포함
	4.	모든 청크를 리스트로 반환
→ 원본 내용 손실 없음 -->
1️⃣ Length-based Sliding Window (길이 기반 + Overlap)

→ 지금 네가 적용한 방식. 의학/논문형 텍스트에서 손실 없는 최강 기본기

✔ 특징
	•	줄바꿈/문단 구조가 엉망이거나 아예 없어도 안정적으로 작동
	•	원본 손실 없음 (슬라이딩 윈도우로 모든 범위 커버)
	•	RAG 검색 안정성 매우 좋음
	•	구현 단순 + 전처리 속도 빠름

✔ 언제 좋음
	•	의학 증례 / 연구 보고서 / 논문형 텍스트
	•	OCR / 웹크롤링처럼 문단 구조가 불명확한 텍스트
	•	“지금 네 데이터 그대로”

✔ 단점
	•	overlap 때문에 청크 수가 늘어날 수 있음
	•	문장 중간에서 끊길 수 있음

max_chunk_chars=1200, overlap=200일 때:
	•	CHUNK 0: 0 ~ 1200
	•	CHUNK 1: 1000 ~ 2200
	•	CHUNK 2: 2000 ~ 3200
	•	CHUNK 3: 3000 ~ 4526

	문단 길이 = 2000자
max = 800
overlap = 200

CHUNK1: 0~800
CHUNK2: 600~1400
CHUNK3: 1200~2000

	•	보통 overlap_chars는 max_chunk_chars의 10~30% 정도가 무난:
	•	1200이면 150~300 정도


In [23]:
import os
import glob
import json
from pathlib import Path
import re
from typing import List, Dict, Iterable

# ========== 0. 데이터 경로 (너의 실제 경로) ==========
DATA_DIR = Path("/Users/test/응 과제해/자연어처리/rag/data")

# ========== 1. JSON 읽기 ==========
def _iter_json(file_path: str) -> Iterable[Dict]:
    with open(file_path, "r", encoding="utf-8") as f:
        try:
            obj = json.load(f)
        except json.JSONDecodeError:
            return
    if isinstance(obj, dict):
        yield obj
    elif isinstance(obj, list):
        for o in obj:
            if isinstance(o, dict):
                yield o


# ========== 2. 손실 없는 청킹 함수 (overlap 포함) ==========
def chunk_v2_overlap(text, max_chunk_chars=1200, min_len=30, overlap_chars=200):
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    raw_chunks = [c.strip() for c in re.split(r"\n{2,}", text)]

    clean = []
    for c in raw_chunks:
        if not c or len(c) < min_len:
            continue

        if len(c) <= max_chunk_chars:
            clean.append(c)
            continue

        step = max_chunk_chars - overlap_chars
        start = 0
        while start < len(c):
            end = min(start + max_chunk_chars, len(c))
            chunk = c[start:end].strip()
            if len(chunk) >= min_len:
                clean.append(chunk)
            if end >= len(c):
                break
            start += step
    return clean

# ========== 3. 저장 폴더 준비 ==========
SAVE_DIR = Path("./chunk_preview_output2")
SAVE_DIR.mkdir(exist_ok=True)

def save_text(filename: str, content: str):
    path = SAVE_DIR / filename
    with open(path, "w", encoding="utf-8") as f:
        f.write(content)
    return str(path)


# ========== 4. 전/후 비교 + 저장 함수 ==========
def debug_preview_and_save2(
    data_dir: Path = DATA_DIR,
    text_field: str = "disease",
    max_chunk_chars: int = 1200,
    min_len: int = 30,
    overlap_chars: int = 200,
    max_files: int = 2,
    max_chunks_per_file: int = 3,
):
    pattern = str(data_dir / "**" / "*.json")
    file_paths = glob.glob(pattern, recursive=True)

    print(f"[debug] JSON 파일 {len(file_paths)}개 중 {max_files}개 미리보기\n")

    for fp in file_paths[:max_files]:
        print("#" * 80)
        print(f"[FILE] {fp}")

        for obj in _iter_json(fp):
            text = str(obj.get(text_field, "")).strip()
            title = obj.get("title", "N/A")
            dept  = obj.get("department", "N/A")

            print(f"  - title      : {title}")
            print(f"  - department : {dept}")
            print(f"  - text len   : {len(text)} chars")

            if not text:
                print("  (⚠️ disease 비어 있음)\n")
                continue

            # 저장: 원본
            original_fname = Path(fp).stem + "_original.txt"
            save_text(original_fname, text)

            # 청킹 수행
            chunks = chunk_v2_overlap(
                text,
                max_chunk_chars=max_chunk_chars,
                min_len=min_len,
                overlap_chars=overlap_chars,
            )

            # 저장: 청크
            chunk_paths = []
            for i, ch in enumerate(chunks):
                chunk_name = f"{Path(fp).stem}_chunk_{i}.txt"
                p = save_text(chunk_name, ch)
                chunk_paths.append(p)

            print(f"  → 청크 {len(chunks)}개 생성")
            print(f"  → 원본 저장: {original_fname}")
            print(f"  → 청크 저장: {len(chunk_paths)}개 (예: {chunk_paths[:1]})\n")

            # ───────── Notebook 출력 (전/후 비교) ─────────
            print("-" * 60)
            print("[원본 text] (앞 300자)")
            print(text[:300] + ("...\n" if len(text) > 300 else "\n"))

            print("[청크 0] (앞 300자)")
            if chunks:
                print(chunks[0][:300] + ("...\n" if len(chunks[0]) > 300 else "\n"))

            for i, ch in enumerate(chunks[:max_chunks_per_file]):
                print("-" * 60)
                print(f"[CHUNK {i}] len={len(ch)}")
                print(ch[:400] + ("\n...\n" if len(ch) > 400 else "\n"))

# ========== 5. 실행 ==========
debug_preview_and_save2(
    data_dir=DATA_DIR,
    text_field="disease",
    max_files=2,
    max_chunks_per_file=3,
)

[debug] JSON 파일 212개 중 2개 미리보기

################################################################################
[FILE] /Users/test/응 과제해/자연어처리/rag/data/TS_말뭉치데이터_안과/0e8bca31-e151-11ef-4bb1-00155dced605.json
  - title      : 40 .진돗개에서의 고니오디스게네시스 관련 녹내장
  - department : 안과
  - text len   : 4526 chars
  → 청크 5개 생성
  → 원본 저장: 0e8bca31-e151-11ef-4bb1-00155dced605_original.txt
  → 청크 저장: 5개 (예: ['chunk_preview_output2/0e8bca31-e151-11ef-4bb1-00155dced605_chunk_0.txt'])

------------------------------------------------------------
[원본 text] (앞 300자)
40 .진돗개에서의 고니오디스게네시스 관련 녹내장
10세 된 온전한 암컷 진돗개 한 마리가 1주일 동안 결막 충혈과 눈 분비물의 병력을 가지고 내원했습니다. 왼쪽 눈. 위협 반응, 눈부심 반사, 직접 동공 빛 반사가 없었하였다. 슬릿 램프 생체 현미경 각막 부종, 섬모 홍조, 수성 발적이 드러났하였다. 안구 내압은 68 mmHg였습니다. 이용 가능한 자료에 따르면, 녹내장과 포도막염 진단이 내려졌습니다. 안압 상승에 따른 시신경 손상. 사례 보고 미주리주 콜럼비아 65211, 미국 고니오디즈게네시스(goniodysgenesis), ...

[청크 0] (앞 300자)
40 .진돗개에서의 고니오디스게네시스 관련 녹내장
10세 된 온전한 암컷 진돗개 한 마리가 1주일 동안 결막 충혈과 눈 분비물의 병력을 가지고 내원했습니다. 왼쪽 눈. 위협 반응, 눈부심 반사, 직접 동공 빛 반

2️⃣ Sentence-based Chunking (문장 단위 + 길이 제한)

→ 가장 자연스러운 방식, GPT가 처리할 때 오류 적음

✔ 특징
	•	문장을 경계로 자르기 때문에 의미 흐름이 자연스럽다
	•	KSS / spaCy / NLTK 등 문장 분리기를 사용
	•	“깨끗한 의미 단위” 유지됨 → QA 정확도 ↑

✔ 언제 좋음
	•	자연어 설명이 많은 문서
	•	문장 중심 구조(블로그, 설명서, 리포트, 교과서 등)

✔ 단점
	•	문장 분리가 느릴 수 있음
	•	일부 어려운 문장 구조에서 분리가 어색할 수 있음

3️⃣ Paragraph-based Chunking (빈 줄 기준 문단 청킹)

→ 문단이 깔끔히 정리된 문서에서 가장 좋은 성능

✔ 특징
	•	빈 줄(\n\n) 기준으로 문단을 쪼개기
	•	문단 자체가 의미 덩어리라 RAG 검색 품질 고정
	•	문서 구조가 깨끗한 경우엔 최선
	•	보통 PDF → 텍스트 변환형 문서에서 정답

✔ 언제 좋음
	•	교과서, 보고서, 매뉴얼, 논문 (문단 명확한 경우)

✔ 단점
	•	너의 데이터처럼 문단이 거의 없는 경우 → 1덩어리로 나옴
	•	1문단이 너무 길면 잘라야 함 → 손실 가능성

➡️ 그래서 *“문단 + 길이 기반 조합”*이 자주 쓰임

4️⃣ Header-aware Chunking (제목/소제목 기반 청킹)

→ 문서 구조(섹션)가 확실할 때 최고 품질

✔ 특징
	•	“제목/소제목”을 감지하여 섹션 단위로 먼저 split
	•	예:
	•	### 1. Introduction
	•	### Case 2 – Diagnosis
	•	2) 검사항목
	•	- 증상 등
	•	각 섹션 안에서 다시 길이 기준 슬라이딩 적용
	•	구조 + 내용 둘 다 보존

✔ 언제 좋음
	•	규칙 있는 텍스트 (매뉴얼, 보고서, 연구 문서, 메뉴판 등)
	•	제목 패턴이 명확한 JSON 라벨 형식

✔ 단점
	•	제목 패턴을 맞추지 못하면 split 실패
	•	의학 증례처럼 “문단이 바로 본문으로 시작”하는 케이스는 구조가 덜함